In [49]:
import os
import pandas as pd
from pathlib import Path
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'


In [50]:
# acquire system path of the current file, join it with the related path of the csv file path.
increase_path = "../Data/Increase/etfs-increase-change-in-open-interest-11-14-2023.csv"
decrease_path = "../Data/Decrease/etfs-decrease-change-in-open-interest-11-14-2023.csv"

#read csv files
increase = pd.read_csv(Path(increase_path))
decrease = pd.read_csv(Path(decrease_path))

print("Increase DataFrame column types:")
print(increase.dtypes)

print("\nDecrease DataFrame column types:")
print(decrease.dtypes)



Increase DataFrame column types:
Symbol       object
Price~      float64
Type         object
Strike      float64
Exp Date     object
DTE         float64
Bid         float64
Ask         float64
Last        float64
Volume      float64
Open Int    float64
OI Chg       object
Delta       float64
IV           object
Time         object
dtype: object

Decrease DataFrame column types:
Symbol       object
Price~      float64
Type         object
Strike       object
Exp Date     object
DTE         float64
Bid         float64
Ask         float64
Last        float64
Volume      float64
Open Int    float64
OI Chg       object
Delta       float64
IV           object
Time         object
dtype: object


In [51]:
# Convert 'Strike' column to float64 in the Decrease DataFrame
increase[['OI Chg', 'Strike']] = increase[['OI Chg', 'Strike']].replace({',': '', 'unch': '0', r'\*': ''}, regex=True).astype(float)
decrease[['OI Chg', 'Strike']] = decrease[['OI Chg', 'Strike']].replace({',': '', 'unch': '0', r'\*': ''}, regex=True).astype(float)

#combine increase and decrease data
combine_df = increase.merge(decrease, how= 'outer')

DTE = "max"
#Select the DTE
if DTE == 'min':
    combine_min_DTE = combine_df[combine_df['DTE'] == combine_df['DTE'].min()]
elif DTE == 'max':
    combine_min_DTE = combine_df[combine_df['DTE'] <= combine_df['DTE'].max()]
else:
    combine_min_DTE = combine_df[combine_df['DTE'] <= DTE]

#Clean the NA data
combine_min_DTE = combine_min_DTE.dropna()

# Transfer columns 'OI Chg' and 'Strike' datatype from str to float.
combine_min_DTE[['OI Chg', 'Strike']] = combine_min_DTE[['OI Chg', 'Strike']].replace({',': '', 'unch': '0', r'\*': ''}, regex=True).astype(float)

#Transfer column 'IV'datatype from str to float.
combine_min_DTE['IV'] = combine_min_DTE['IV'].str.replace('%', '').str.replace(',', '').astype(float)/100

#Sorted by orders 'Symbol','Type','Strike','Open Int','Volume','OI Chg','IV'.
combine_sort_df = combine_min_DTE.sort_values(['Symbol','Type','Strike','Open Int','Volume','OI Chg','IV'])

# Rename columns
combine_sort_df.columns = ['Symbol', 'Price', 'Type', 'Strike', 'Exp Date', 'DTE', 'Bid', 'Ask', 'Last', 'Volume', 'Open Int', 'OI Chg', 'Delta', 'IV', 'Time']

# Calculate 'Midpoint' and add the column
combine_sort_df['Midpoint'] = (combine_sort_df['Ask'] + combine_sort_df['Bid']) / 2

# Reorder columns
column_order = ['Symbol', 'Price', 'Type', 'Strike', 'Exp Date', 'DTE', 'Bid', 'Midpoint', 'Ask', 'Last', 'Volume', 'Open Int', 'OI Chg', 'Delta', 'IV', 'Time']
combine_sort_df = combine_sort_df[column_order]




# return data
combine_sort_df

,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,OI Chg,Delta,IV,Time
56,ARKG,25.94,Call,24.5,2023-11-17,3.0,1.30,1.350,1.40,1.25,2012.0,5005.0,4921.0,0.851136,0.5404,13:10 ET
2155,ARKG,25.94,Call,25.0,2023-11-17,3.0,0.85,0.925,1.00,1.00,1655.0,1124.0,5.0,0.718221,0.5932,11:18 ET
1162,ARKG,25.94,Call,28.0,2023-12-15,31.0,0.40,0.450,0.50,0.40,3527.0,3820.0,-263.0,0.250892,0.3754,14:33 ET
1040,ARKG,25.94,Put,23.5,2023-12-01,17.0,0.15,0.200,0.25,0.20,2000.0,4191.0,-1858.0,-0.151290,0.4395,09:33 ET
2151,ARKG,25.94,Put,24.0,2023-12-01,17.0,0.25,0.300,0.35,0.25,1001.0,1009.0,5.0,-0.192689,0.4059,11:13 ET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,XRT,62.53,Put,62.0,2023-11-17,3.0,0.45,0.475,0.50,0.48,1345.0,1694.0,-11.0,-0.386112,0.3084,15:17 ET
1870,XRT,62.53,Put,63.0,2023-12-15,31.0,1.81,1.830,1.85,1.97,969.0,1461.0,0.0,-0.522794,0.2279,14:22 ET
1754,YANG,10.59,Call,8.0,2024-01-19,66.0,2.80,2.850,2.90,3.09,700.0,548.0,0.0,0.893238,0.6180,11:56 ET
2307,YANG,10.59,Put,11.0,2023-12-15,31.0,1.20,1.225,1.25,1.20,569.0,133.0,14.0,-0.507972,0.8137,14:39 ET


In [52]:
import sys
import os
import psycopg2
import psycopg2.extras
import pandas as pd
import pytz
from datetime import datetime
import traceback

# Load and assign variables.

SQL_DB = ''
SQL_USER = ''
SQL_PASSWORD = ''
SQL_HOST = ''

def connect_data_base(database=SQL_DB, user=SQL_USER, password=SQL_PASSWORD, host=SQL_HOST):
    con = psycopg2.connect(database=SQL_DB, user=SQL_USER, password=SQL_PASSWORD, host=SQL_HOST)
    cur = con.cursor()
    cur.execute('SELECT version()')
    version = cur.fetchone()[0]
    print(f"DataBaseFlow Connected!\nVersion: {version}")
    return con, cur

date = "11-14-2023"

now_timestamp = int(datetime.now().timestamp()) #时间戳只保留秒（去掉更低的精度，保留整数部分）
date_parts = date.split('-')  #分割日期字符串
con, cur = connect_data_base() #连接数据库



DataBaseFlow Connected!
Version: PostgreSQL 14.8 on aarch64-unknown-linux-musl, compiled by gcc (Alpine 12.2.1_git20220924-r10) 12.2.1 20220924, 64-bit


In [53]:
combine_data = combine_sort_df

# 自定义函数，根据"Last"和"Midpoint"的关系判断"sentiment"
def get_initiator(row):
    if row['Last'] < row['Bid']:
        return 'Aggressive Seller'
    elif row['Last'] > row['Ask']:
        return 'Aggressive Buyer'
    elif row['Last'] < row['Midpoint']:
        return 'Seller'
    elif row['Last'] > row['Midpoint']:
        return 'Buyer'
    else:
        return 'Neutral'
# Add "Initiator" column
combine_data['Initiator'] = combine_data.apply(get_initiator, axis=1)

#重新排列，把'Initiator'放在'Last'后：
new_order = ['Symbol', 'Price', 'Type', 'Strike', 'Exp Date', 'DTE', 'Bid', 'Midpoint', 'Ask', 'Last', 'Initiator', 'Volume', 'Open Int', 'OI Chg', 'Delta', 'IV', 'Time']

# 使用reindex()方法重新排列列的顺序
combine_data = combine_data.reindex(columns=new_order)

#构建写操作的表名
write_table_name = f"_{date_parts[2]}_{date_parts[0]}_{date_parts[1]}_etfs_{now_timestamp}"  # 按照'YYYY_MM_DD'格式重新排列日期部分

# Delete old data from the table if it exists
delete_query = f"DROP TABLE IF EXISTS {write_table_name}"
cur.execute(delete_query)
con.commit()

# Create table if not exists
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {write_table_name} (
    "Symbol" TEXT,
    "Price" NUMERIC,
    "Type" TEXT,
    "Strike" NUMERIC,
    "Exp Date" TEXT,
    "DTE" NUMERIC,
    "Bid" NUMERIC,
    "Midpoint" NUMERIC,
    "Ask" NUMERIC,
    "Last" NUMERIC,
    "Initiator" TEXT,
    "Volume" NUMERIC,
    "Open Int" NUMERIC,
    "OI Chg" NUMERIC,
    "Delta" NUMERIC,
    "IV" NUMERIC,
    "Time" TEXT
)
"""
cur.execute(create_table_query)
con.commit()

# 批量插入数据
insert_query = f"INSERT INTO {write_table_name} VALUES %s"
rows = [tuple(row) for _, row in combine_data.iterrows()]
psycopg2.extras.execute_values(cur, insert_query, rows)
con.commit()
print("Data inserted successfully!")
print(f"{write_table_name}")

Data inserted successfully!
_2023_11_14_etfs_1700038878
